# 市区町村一覧とその緯度経度

データ出典元: [「位置参照情報ダウンロードサービス」（国土交通省）](https://nlftp.mlit.go.jp/index.html) => `../data/mlit/`

In [1]:
!tree ../data/mlit/ | head

../data/mlit/
├── 01000-14.0b
│   ├── 01_2020.csv
│   ├── META_01_2020.xml
│   └── format_2020.html
├── 02000-14.0b
│   ├── 02_2020.csv
│   ├── META_02_2020.xml
│   └── format_2020.html
├── 03000-14.0b


---

In [2]:
from pathlib import Path
from statistics import mean
from collections import defaultdict
import json

import pandas as pd

In [3]:
df_list = []
for fpath in sorted(Path("../data/mlit/").glob("*/*.csv")):
    df_list.append(
        pd.read_csv(fpath, encoding="cp932")
    )
assert len(df_list) == 47
    
df_all = pd.concat(df_list).sort_values(["都道府県コード", "市区町村コード", "大字町丁目コード"]).reset_index(drop=True)
df_all

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,1,北海道,1101,札幌市中央区,11010001001,旭ケ丘一丁目,43.042230,141.319722,0,3
1,1,北海道,1101,札幌市中央区,11010001002,旭ケ丘二丁目,43.039768,141.321733,0,3
2,1,北海道,1101,札幌市中央区,11010001003,旭ケ丘三丁目,43.039569,141.319617,0,3
3,1,北海道,1101,札幌市中央区,11010001004,旭ケ丘四丁目,43.038819,141.323040,0,3
4,1,北海道,1101,札幌市中央区,11010001005,旭ケ丘五丁目,43.036547,141.322217,0,3
...,...,...,...,...,...,...,...,...,...,...
190205,47,沖縄県,47381,八重山郡竹富町,473810010000,字新城,24.234027,123.944947,0,1
190206,47,沖縄県,47381,八重山郡竹富町,473810011000,字崎山,24.305231,123.685666,0,1
190207,47,沖縄県,47381,八重山郡竹富町,473810012000,字鳩間,24.472254,123.820428,0,1
190208,47,沖縄県,47381,八重山郡竹富町,473810013000,字竹富,24.326125,124.089079,0,1


In [4]:
%%time

pref_munic_oaza_coords = defaultdict(lambda: defaultdict(list))

for _, row in df_all.iterrows():
    d = row.to_dict()
    pref_munic_oaza_coords[d["都道府県名"]][d["市区町村名"]].append(
        [d["経度"], d["緯度"]]
    )

CPU times: user 3.69 s, sys: 13.8 ms, total: 3.7 s
Wall time: 3.71 s


In [5]:
pref_munic_coords = defaultdict(dict)
for pref, munic_oaza_coords in pref_munic_oaza_coords.items():
    for munic, oaza_coords in munic_oaza_coords.items():
        # 「全大字の緯度経度の平均」を「市区町村を代表する緯度経度」とする
        munic_lon = mean([x[0] for x in oaza_coords])
        munic_lat = mean([x[1] for x in oaza_coords])
        pref_munic_coords[pref][munic] = [munic_lon, munic_lat]

pd.DataFrame(
    [(pref, len(munic_coords)) for pref, munic_coords in pref_munic_coords.items()],
    columns=["都道府県", "市区町村の数"]
)

,都道府県,市区町村の数
0,北海道,188
1,青森県,40
2,岩手県,33
3,宮城県,39
4,秋田県,25
5,山形県,35
6,福島県,59
7,茨城県,44
8,栃木県,25
9,群馬県,35


In [6]:
with open("../data/pref_munic_coords.json", "w") as fp:
    json.dump(pref_munic_coords, fp, indent=2, ensure_ascii=False)

In [7]:
!head ../data/pref_munic_coords.json

{
  "北海道": {
    "札幌市中央区": [
      141.33714566565658,
      43.05310114646465
    ],
    "札幌市北区": [
      141.33890471901609,
      43.11416385241249
    ],


In [8]:
!tail ../data/pref_munic_coords.json

    "八重山郡竹富町": [
      123.8787486923077,
      24.30438653846154
    ],
    "八重山郡与那国町": [
      122.987678,
      24.455925
    ]
  }
}